<a href="https://colab.research.google.com/github/mohammedalqi/anabul-app/blob/main/Anabul_Dog_Breeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Mendownload dataset dari Kaggle
!pip install kaggle
import os
os.environ['KAGGLE_USERNAME'] = 'your_kaggle_username'
os.environ['KAGGLE_KEY'] = 'your_kaggle_key'
!kaggle datasets download -d yapwh1208/dogs-breed-dataset

Pastikan untuk menggantikan 'your_kaggle_username' dan 'your_kaggle_key' dengan kredensial akun Kaggle Anda. Juga, pastikan bahwa Anda telah menginstal paket Kaggle, yang dapat Anda lakukan dengan menjalankan !pip install kaggle di notebook Anda sebelum menjalankan kode ini.

In [ ]:
# Ekstraksi dataset
import zipfile
with zipfile.ZipFile('dogs-breed-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('dog_breeds_dataset')

In [ ]:
# Mendefinisikan path dataset
train_data_dir = 'dog_breeds_dataset/Train'
validation_data_dir = 'dog_breeds_dataset/Test'

In [ ]:
# Mendefinisikan parameter training
batch_size = 32
epochs = 10
img_width, img_height = 150, 150

In [ ]:
# Membangun model CNN
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(133, activation='softmax')
])

In [ ]:
# Mengkompilasi model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Membuat augmentasi gambar
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
# Mempersiapkan data latih
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
# Mempersiapkan data validasi
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
# Melatih model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

In [ ]:
# Menyimpan model
model.save('dog_breed_classification_model.h5')